In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("AppName").getOrCreate()

In [3]:
states = {"NY":"New York", "CA":"California", "FL":"Florida"}
broadcastStates = spark.sparkContext.broadcast(states)

In [5]:
data = [
    ("James", "Smith", "USA", "CA"),
    ("Michael", "Rose", "USA", "NY"),
    ("Robert", "Williams", "USA", "CA"),
    ("Maria", "Jones", "USA", "FL")
]

In [6]:
columns = ["firstname", "lastname", "country", "state"]

In [7]:
df = spark.createDataFrame(data=data, schema=columns)
df.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)



In [8]:
def state_convert(code):
    return broadcastStates.value[code]

In [12]:
results = df.rdd.map(lambda x: (x[0], x[1], x[2], state_convert(x[3]))).toDF(columns)
results.show(truncate=False)

In [14]:
filterDF = df.where((df['state'].isin(broadcastStates.value)))